# UMAPによる次元削減（50 → 2）

## 1. ライブラリのインポート

In [1]:
import numpy as np
from cuml import UMAP
import os
from tqdm import tqdm

## 2. データの読み込み

In [2]:
model_name = "esm1b"

pca_dir = f"../data/embedding-vectors/{model_name}/pca50"  # PCA50データの読み込み

files = [os.path.join(pca_dir, f) for f in os.listdir(pca_dir) if f.endswith(".npy")]  # .npy ファイルを全て取得
files.sort()  # 一応ソート

### 2.1 データを（ランダムに）抽出（件数を選んだりもできる）

In [25]:
sample = []

for filename in tqdm(files):
    arr = np.load(filename, mmap_mode="r")
    m = arr.shape[0]

    # ランダムに 100件 抽出するやつ
    # k = min(100, m)
    k = m
    idxs = np.random.choice(m, size=k, replace=False)

    sample.append(arr[idxs])

100%|█████████████████████████████████████████████████████████████████████████████| 7716/7716 [00:01<00:00, 5216.62it/s]


### 2.2 UMAP で次元削減

In [29]:
sample = np.vstack(sample)
reducer = UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(sample)

[2025-10-09 13:01:53.797] [CUML] [info] build_algo set to brute_force_knn because random_state is given


## 3. 各 seq ファイルごとに transform して保存

In [30]:
output_dir = f"../data/embedding-vectors/{model_name}/umap2"
os.makedirs(output_dir, exist_ok=True)

for filename in tqdm(files):
    arr50 = np.load(filename, mmap_mode="r")
    coords2 = reducer.transform(arr50)

    save_filename = os.path.basename(filename).replace("pca50", "umap2")
    np.save(os.path.join(output_dir, save_filename), coords2)

100%|███████████████████████████████████████████████████████████████████████████████| 7716/7716 [14:49<00:00,  8.67it/s]
